In [ ]:

# # ADD1) jigsaw-toxic-severity-rating
# # ADD2) jigsaw-toxic-comment-classification-challenge

# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### libraries

In [ ]:
!pip install nltk
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk 
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import re
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob 
from sklearn.feature_extraction.text import TfidfVectorizer
#!pip install xgboost
from xgboost import XGBRegressor 

submit - sample code
- https://www.kaggle.com/kush1729/xgboostregressor-with-optuna

In [ ]:
dirname = '/kaggle/input/jigsaw-toxic-severity-rating'
# submission = pd.read_csv(f"{dirname}/sample_submission.csv")
# df_val = pd.read_csv(f"{dirname}/validation_data.csv")
data = pd.read_csv(f"{dirname}/comments_to_score.csv")


# https://www.kaggle.com/samarthagarwal23/mega-b-ridge-to-the-top-lb-0-85
dirname = '/kaggle/input/jigsaw-toxic-comment-classification-challenge'
train_data = pd.read_csv(f"{dirname}/train.csv")
text = data['text']
# df_test = pd.read_csv(f"{dirname}/test.csv")
# df_test_l = pd.read_csv(f"{dirname}/test_labels.csv")

In [ ]:
data.head()

In [ ]:
train_data.head()

In [ ]:
%%time
score = []
for i in range(len(train_data)):
    x = train_data.iloc[i]
    s = sum(x['toxic':].values) # 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'
    score.append(round(s/6, 3))

In [ ]:
%%time
train_data['score'] = score
train_data.head()

In [ ]:
#Stopwords
import nltk
nltk.download('stopwords')
stop = set(stopwords.words('english'))

In [ ]:
def lowercase(te):
    tmp = []
    #Complete 
    for i in te : 
        tmp.append(i.lower())
    return tmp
def remove_symbols(text):
    #commplete
    tmp = []
    for i in text: 
        #i = re.sub(r'\n','',i)
       # i = re.sub(r'""', ' ',i)
        i = re.sub(r'[^\w]',' ',i) #Remove all types of symbols from string
        tmp.append(i.replace(') ',''))
    return tmp
def remove_stopwords(text): 
    tmp = []
    for i in text: 
        te = str(i)
        words = word_tokenize(te)
        token_without = [word for word in words if not word in stop]
        s = ''
        for w in token_without : 
            s = s + w + ' '
        tmp.append(s)
    return tmp
def stemming(text): 
    tmp = []
    for i in text : 
        te = str(i)
        lemmatizer = WordNetLemmatizer()
        words = word_tokenize(te)
        s = ''
        for w in words:
            rw = lemmatizer.lemmatize(w)
            s = s + rw + ' '
        tmp.append(s)
    return tmp
def stemming_lem_stop(text):
    tmp = []
    for i in text:
        te = str(i)
        words = word_tokenize(te)
        ps = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        token_without = [word for word in words if not word in stop] #Remove Stopwords from token
        s = ''
        for w in token_without: 
            rootword = ps.stem(w)
            rw = lemmatizer.lemmatize(rootword) #Lemmatization
            s = s + rw + ' '
        tmp.append(s)
    return tmp

In [ ]:
%%time
train1 = lowercase(train_data['comment_text'])
train2 = remove_symbols(train1)
train4 = stemming(train2)
train3 = remove_stopwords(train4)
test1 = lowercase(text)
test2 = remove_symbols(test1)
test4 = stemming(test2)
test3 = remove_stopwords(test4)

In [ ]:
## Training set

%%time
pol = []
for i in train3:
    analysis = TextBlob(i)
    x = analysis.sentiment.polarity
    NewValue = (((x - (-1)) * 1) / 2) + 0 #NewValue = (((OldValue - OldMin) * NewRange) / OldRange) + NewMin
    if NewValue == 0.5 :
        pol.append(0)
    else : 
        pol.append(round(NewValue,2))

In [ ]:
len(train3)

In [ ]:
valid_x = train3[120000:]
valid_y = pol[120000:]

In [ ]:
com = []
di = {}
pol1 = []
for i in range(len(data['comment_id'])): 
    com.append(data['comment_id'][i])

In [ ]:
tfid = TfidfVectorizer(max_features = 10)
res = tfid.fit_transform(train3).toarray()
res1 = tfid.fit_transform(test3).toarray()
valid_res = tfid.fit_transform(valid_x).toarray()
# dic = {}
# for e1, e2 in zip(tfid.get_feature_names(), tfid.idf_): 
#     dic[e1] = e2

In [ ]:
%%time
import xgboost as xgb 
import optuna 
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)
import sklearn.metrics
def objective(trial):
    dtrain = xgb.DMatrix(res, label=pol)
    dvalid = xgb.DMatrix(valid_res, label=valid_y)
    
    param = {
        #'silent': 1,
        'n_estimators' : trial.suggest_int('n_estimators',10,1000, log=True),
        'max_depth':trial.suggest_int('max_depth',2,32, log= True),
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        #'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 1, log=True)
    }

    bst = xgb.train(param,dtrain)
    preds = bst.predict(dvalid)
    #pred_labels = np.rint(preds)
    accuracy = sklearn.metrics.mean_absolute_error(valid_y, preds)
    
    return accuracy 

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
trial = study.best_trial
val = []
for key, value in trial.params.items():
    val.append(value)
    print("    {}: {}".format(key, value))

In [ ]:
model = XGBRegressor(n_estimators = val[0],max_depth = val[1] ,booster = val[2], eta = val[3])
model.fit(res,pol)
y_pred = abs(model.predict(res1))

In [ ]:
di = {'comment_id': com, 'score': y_pred}
df = pd.DataFrame(di)

In [ ]:
#Output file
df.to_csv("submission.csv", index=False)

In [ ]:
df